# Importing Packages

In [82]:
import pandas as pd
import numpy as np
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, OneHotEncoder, PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import r2_score, root_mean_squared_error
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import cross_val_score



# Read in / Partition Data

In [83]:
ames = pd.read_csv("C:/Users/petew/OneDrive/Desktop/Cal Poly/MSBA/GSB-544-(Python)/Week-7/AmesHousing.csv")
ames.head()

,Order,PID,MS SubClass,MS Zoning,Lot Frontage,Lot Area,Street,Alley,Lot Shape,Land Contour,Utilities,Lot Config,Land Slope,Neighborhood,Condition 1,Condition 2,Bldg Type,House Style,Overall Qual,Overall Cond,Year Built,Year Remod/Add,Roof Style,Roof Matl,Exterior 1st,Exterior 2nd,Mas Vnr Type,Mas Vnr Area,Exter Qual,Exter Cond,Foundation,Bsmt Qual,Bsmt Cond,Bsmt Exposure,BsmtFin Type 1,BsmtFin SF 1,BsmtFin Type 2,BsmtFin SF 2,Bsmt Unf SF,Total Bsmt SF,Heating,Heating QC,Central Air,Electrical,1st Flr SF,2nd Flr SF,Low Qual Fin SF,Gr Liv Area,Bsmt Full Bath,Bsmt Half Bath,Full Bath,Half Bath,Bedroom AbvGr,Kitchen AbvGr,Kitchen Qual,TotRms AbvGrd,Functional,Fireplaces,Fireplace Qu,Garage Type,Garage Yr Blt,Garage Finish,Garage Cars,Garage Area,Garage Qual,Garage Cond,Paved Drive,Wood Deck SF,Open Porch SF,Enclosed Porch,3Ssn Porch,Screen Porch,Pool Area,Pool QC,Fence,Misc Feature,Misc Val,Mo Sold,Yr Sold,Sale Type,Sale Condition,SalePrice
0,1,526301100,20,RL,141.0,31770,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,5,1960,1960,Hip,CompShg,BrkFace,Plywood,Stone,112.0,TA,TA,CBlock,TA,Gd,Gd,BLQ,639.0,Unf,0.0,441.0,1080.0,GasA,Fa,Y,SBrkr,1656,0,0,1656,1.0,0.0,1,0,3,1,TA,7,Typ,2,Gd,Attchd,1960.0,Fin,2.0,528.0,TA,TA,P,210,62,0,0,0,0,NaN,NaN,NaN,0,5,2010,WD,Normal,215000
1,2,526350040,20,RH,80.0,11622,Pave,NaN,Reg,Lvl,AllPub,Inside,Gtl,NAmes,Feedr,Norm,1Fam,1Story,5,6,1961,1961,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,CBlock,TA,TA,No,Rec,468.0,LwQ,144.0,270.0,882.0,GasA,TA,Y,SBrkr,896,0,0,896,0.0,0.0,1,0,2,1,TA,5,Typ,0,NaN,Attchd,1961.0,Unf,1.0,730.0,TA,TA,Y,140,0,0,0,120,0,NaN,MnPrv,NaN,0,6,2010,WD,Normal,105000
2,3,526351010,20,RL,81.0,14267,Pave,NaN,IR1,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,6,6,1958,1958,Hip,CompShg,Wd Sdng,Wd Sdng,BrkFace,108.0,TA,TA,CBlock,TA,TA,No,ALQ,923.0,Unf,0.0,406.0,1329.0,GasA,TA,Y,SBrkr,1329,0,0,1329,0.0,0.0,1,1,3,1,Gd,6,Typ,0,NaN,Attchd,1958.0,Unf,1.0,312.0,TA,TA,Y,393,36,0,0,0,0,NaN,NaN,Gar2,12500,6,2010,WD,Normal,172000
3,4,526353030,20,RL,93.0,11160,Pave,NaN,Reg,Lvl,AllPub,Corner,Gtl,NAmes,Norm,Norm,1Fam,1Story,7,5,1968,1968,Hip,CompShg,BrkFace,BrkFace,NaN,0.0,Gd,TA,CBlock,TA,TA,No,ALQ,1065.0,Unf,0.0,1045.0,2110.0,GasA,Ex,Y,SBrkr,2110,0,0,2110,1.0,0.0,2,1,3,1,Ex,8,Typ,2,TA,Attchd,1968.0,Fin,2.0,522.0,TA,TA,Y,0,0,0,0,0,0,NaN,NaN,NaN,0,4,2010,WD,Normal,244000
4,5,527105010,60,RL,74.0,13830,Pave,NaN,IR1,Lvl,AllPub,Inside,Gtl,Gilbert,Norm,Norm,1Fam,2Story,5,5,1997,1998,Gable,CompShg,VinylSd,VinylSd,NaN,0.0,TA,TA,PConc,Gd,TA,No,GLQ,791.0,Unf,0.0,137.0,928.0,GasA,Gd,Y,SBrkr,928,701,0,1629,0.0,0.0,2,1,3,1,TA,6,Typ,1,TA,Attchd,1997.0,Fin,2.0,482.0,TA,TA,Y,212,34,0,0,0,0,NaN,MnPrv,NaN,0,3,2010,WD,Normal,189900


In [84]:
X = ames.drop("SalePrice", axis = 1)
y = ames["SalePrice"]



X_train, X_test, y_train, y_test = train_test_split(X, y)

# Size and Rooms

In [96]:
from sklearn.compose import ColumnTransformer

ct = ColumnTransformer(
    [
        ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"])
    ],
  remainder = "drop"
).set_output(transform = "pandas")

lr_pipeline = Pipeline(
    [("preprocessing", ct),
    ("linear_regression", LinearRegression())]
).set_output(transform = "pandas")

lr_pipeline.fit(X_train, y_train)
y_pred = lr_pipeline.predict(X_test)

print(f"R-Squared = {round(r2_score(y_test, y_pred), 3)}")
print(f"RMSE = {round(root_mean_squared_error(y_test, y_pred))}")

# print(lr_pipeline.named_steps["linear_regression"].intercept_)
# print(lr_pipeline.named_steps["linear_regression"].coef_)

R-Squared = 0.476
RMSE = 55740
R-Squared = 0.476
RMSE = 55740


# Size Rooms and Building Type

In [95]:
ct = ColumnTransformer(
    [
        ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"]),
        ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"])
    ],
  remainder = "drop"
).set_output(transform = "pandas")

lr_pipeline = Pipeline(
    [("preprocessing", ct),
    ("linear_regression", LinearRegression())]
).set_output(transform = "pandas")

lr_pipeline.fit(X_train, y_train)
y_pred = lr_pipeline.predict(X_test)

print(f"R-Squared = {round(r2_score(y_test, y_pred), 3)}")
print(f"RMSE = {round(root_mean_squared_error(y_test, y_pred))}")

R-Squared = 0.518
RMSE = 53458
R-Squared = 0.518
RMSE = 53458


# Size, Building Type and Interaction Terms

In [108]:
ct_dummify = ColumnTransformer(
    [
        ("dummify", OneHotEncoder(sparse_output = False, handle_unknown="ignore"), ["Bldg Type"]),
        ("standardize", StandardScaler(), ["Gr Liv Area"])
    ],
  remainder = "drop"
).set_output(transform = "pandas")

ct_interaction = ColumnTransformer(
    [
        ("interaction", PolynomialFeatures(interaction_only= True), ['dummify__Bldg Type_1Fam', 'dummify__Bldg Type_2fmCon', 
          'dummify__Bldg Type_Duplex', 'dummify__Bldg Type_Twnhs', 
          'dummify__Bldg Type_TwnhsE', 'standardize__Gr Liv Area'])
    ],
  remainder = "drop"
).set_output(transform = "pandas")

lr_pipeline = Pipeline(
    [("dummify_standardize", ct_dummify),
    ("interaction", ct_interaction),
    ("linear_regression", LinearRegression())]
).set_output(transform = "pandas")

lr_pipeline.fit(X_train, y_train)
y_pred = lr_pipeline.predict(X_test)

print(f"R-Squared = {round(r2_score(y_test, y_pred), 3)}")
print(f"RMSE = {round(root_mean_squared_error(y_test, y_pred))}")


R-Squared = 0.544
RMSE = 52007
R-Squared = 0.544
RMSE = 52007


In [109]:
ct_dummify = ColumnTransformer(
    [
        ("dummify", OneHotEncoder(sparse_output = False), ["Bldg Type"]),
        ("standardize", StandardScaler(), ["Gr Liv Area", "TotRms AbvGrd"])
    ],
  remainder = "drop"
).set_output(transform = "pandas")

ct_interaction = ColumnTransformer(
    [
        ("poly_size", PolynomialFeatures(degree=5, include_bias=False), ["standardize__Gr Liv Area"]),
        ("poly_rooms", PolynomialFeatures(degree=5, include_bias=False), ["standardize__TotRms AbvGrd"])
    ],
  remainder = "passthrough"
).set_output(transform = "pandas")


lr_pipeline = Pipeline(
    [("dummify_standardize", ct_dummify),
    ("interaction", ct_interaction),
    ("linear_regression", LinearRegression())]
).set_output(transform = "pandas")

lr_pipeline.fit(X_train, y_train)
y_pred = lr_pipeline.predict(X_test)

print(f"R-Squared = {round(r2_score(y_test, y_pred), 3)}")
print(f"RMSE = {round(root_mean_squared_error(y_test, y_pred))}")


R-Squared = 0.547
RMSE = 51847
R-Squared = 0.547
RMSE = 51847
